# EVAL


import plotly.express as px
import pandas as pd
import yaml
from unidecode import unidecode  
from modeling_utils.modeling_utils import *
import pandas as pd
import pickle
import os

In [59]:
file_path_yaml = os.path.join(os.getcwd(), 
                        'config',
                        'config.yaml')

with open(file_path_yaml, 'r') as file:
        yaml_data = yaml.load(file, Loader=yaml.FullLoader)

DEBUG = False

# Incorporate data
df = pd.read_csv(yaml_data['PATH_TO_CSV'],
                encoding="utf-8" , 
                nrows=yaml_data['N_ROWS'])   

file_path_vectorized_c = os.path.join(os.getcwd(), 
                                        'csv',
                                        'vectorized_corpus.csv')
print(file_path_vectorized_c)

file_path_kmeans = os.path.join(os.getcwd(), 
                        'csv',
                        'kmeans_clusters.csv')

vectorized_corpus =pd.read_csv(file_path_vectorized_c)
kmeans_clusters = pd.read_csv(file_path_kmeans)


df['first_two_digits_code'] = df['agileitemsmp_id'].apply(lambda x: str(x)[:2])
df['feature_vector']=  df['first_two_digits_code'] + ' '+ df['agileoffereditemsdescripcionofertada']
corpus = df['feature_vector'].apply(lambda x: unidecode(x).lower())

file_path_vectorizer = os.path.join(os.getcwd(), 
                                'models',
                                'count_vectorized_model.pkl')

with open(file_path_vectorizer, 'rb') as model_file:
    vectorizer = pickle.load(model_file)


c:\Proyectos Git\Universidad\licitalabclon\MDS_licitalab\Modeling\ProviderDescriptionBased\csv\vectorized_corpus.csv


TypeError: 'CountVectorizer' object is not callable

In [67]:
rut = df['taxnumberprovider'].sample(n = 1).values
 
user = str(rut[0])
print(user)

77.229.656-8


In [68]:

example_user_df = df.query(f'taxnumberprovider == "{user}"') # and adjudicada == True' ) #no es necesario que la licitacion este adjudicada, pero aquí probamos con licitaciones adjudicadas

if yaml_data['USE_AWARDED']:
    example_user_df = example_user_df.query(f'adjudicada == True' )

cluster_number = predict_user_item_cluster(example_user_df,
                                        vectorizer,
                                        vectorized_corpus,
                                        yaml_data['ITEMS_TO_CONSIDER'],
                                        kmeans_clusters)
#matching 
gg = match_cluster_data_with_agilebuy(_user_df = example_user_df,
                                    _df = df,
                                    _vectorizer = vectorizer,
                                    _vectorized_data= vectorized_corpus,
                                    _items_to_consider=yaml_data['ITEMS_TO_CONSIDER'],
                                    _cluster_method=kmeans_clusters)


Unseen data point belongs to cluster 10
Unseen data point belongs to cluster 10


In [69]:
yaml_data

{'DEBUG': True,
 'PATH_TO_CSV': 'C:\\Proyectos Git\\Universidad\\licitalabclon\\MDS_licitalab\\Modeling\\20231007200103_query_results.csv',
 'N_ROWS': 500000,
 'ELBOW_MIN': 20,
 'ELBOW_MAX': 100,
 'ELBOW_N': 50,
 'ITEMS_TO_CONSIDER': 15,
 'USE_AWARDED': False,
 'CLUSTERIZATION_TYPE': 'kmeans'}

In [70]:
exploration_query = gg.query(f"Cluster == {cluster_number} and taxnumberprovider == '{user}'") 

save = exploration_query.groupby(by = 'first_two_digits_code').count()['agileitemsproductcategory'].reset_index()
    
fig = px.bar(save, x='first_two_digits_code', y='agileitemsproductcategory', title='Categorías Presentes')
data = exploration_query.groupby(by=['agilebuyingsdescription','agileitemsproductcategory','adjudicada','taxnumberprovider']).count().reset_index()
data = data[['taxnumberprovider','agilebuyingsdescription','agileitemsproductcategory','adjudicada']]
display(data)
 

,taxnumberprovider,agilebuyingsdescription,agileitemsproductcategory,adjudicada


In [71]:

grouped = gg.groupby(by =['taxnumberprovider','agilebuyingsdescription','adjudicada']).agg({'agilebuyingsdescription':'nunique'})
grouped = grouped.assign(Count=gg.groupby(by=['taxnumberprovider','agilebuyingsdescription']).agg({'adjudicada':'count'}))
 
display(grouped) 

agilebuyingsdescription  \
taxnumberprovider agilebuyingsdescription                            adjudicada                            
10.011.441-0      SE REQUIERE MATERIALES ELECTRICOS PARA LICEO DE... False                             1   
10.016.503-1      Adquisición de carne que serán empleados en el ... True                              1   
10.018.801-5      Se solicita cotizar según detalles de los produ... False                             1   
10.025.673-8      Adquisición de televisor SMART TV para reposici... False                             1   
                  SE SOLICITA ADJUNTAR  FICHA CON LAS ESPECIFICAC... False                             1   
...                                                                                                  ...   
99.593.170-2      Se necesita adquirir 2.200 unidades de cánulas ... False                             1   
                  Se requiere adquirir insumo ALARGADOR VENOSO DE... False                             1   
                  TET ANILLADO CON ESPIRAL 5,5 MM DESECHABLE (FAV... False                             1   
                  adjuntar cotización indicando tiempos de entrega   False                             1   
                  vencimiento igual o superior a 12 meses(si apli... False                             1   

                                                                                 Count  
taxnumberprovider agilebuyingsdescription                            adjudicada         
10.011.441-0      SE REQUIERE MATERIALES ELECTRICOS PARA LICEO DE... False           1  
10.016.503-1      Adquisición de carne que serán empleados en el ... True            1  
10.018.801-5      Se solicita cotizar según detalles de los produ... False           1  
10.025.673-8      Adquisición de televisor SMART TV para reposici... False           1  
                  SE SOLICITA ADJUNTAR  FICHA CON LAS ESPECIFICAC... False           1  
...                                                                                ...  
99.593.170-2      Se necesita adquirir 2.200 unidades de cánulas ... False           1  
                  Se requiere adquirir insumo ALARGADOR VENOSO DE... False           1  
                  TET ANILLADO CON ESPIRAL 5,5 MM DESECHABLE (FAV... False           1  
                  adjuntar cotización indicando tiempos de entrega   False           1  
                  vencimiento igual o superior a 12 meses(si apli... False           1  

[14486 rows x 2 columns]